In [ ]:
# Function to save velocity with arm score plots
def plot_velocity_with_arm_score(df, athlete_id, pdf):
    athlete_data = df[df['ID'] == athlete_id]
    armcare_data = athlete_data.dropna(subset=['Arm Score'])

    # Extract all relevant pitching test types, including Mound Velo & Catchers Velo
    pitching_data = athlete_data[
        (athlete_data['Workout Type'] == 'Pitching') & 
        (athlete_data['Test Sub-Type'] == '5oz')
    ]
    trackman_data = athlete_data[
        (athlete_data['Test Type'] == 'Trackman Bullpen') & 
        (athlete_data['Test Sub-Type'] == 'Fastball')
    ]

    # Parse dates and sort
    for data in [armcare_data, pitching_data, trackman_data]:
        if not data.empty:
            data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
            data.dropna(subset=['Date'], inplace=True)
            data.sort_values('Date', inplace=True)

    # Ensure valid armcare data
    if not armcare_data.empty:
        armcare_data['Arm Score'] = pd.to_numeric(armcare_data['Arm Score'], errors='coerce')
        armcare_data.dropna(subset=['Arm Score'], inplace=True)

    # **Updated test types to include "Mound Velo" & "Catchers Velo"**
    test_types = ['Roll Ins', 'Double Plays', 'Turn and Burns', 'Pulldowns', 
                  'Trackman Bullpen', 'Mound Velo', 'Catchers Velo']
    colors = ['red', 'green', 'blue', 'purple', 'orange', 'darkcyan', 'brown']
    velocity_changes = []

    for i, test_type in enumerate(test_types):
        if test_type == 'Trackman Bullpen':
            type_data = trackman_data
        else:
            type_data = pitching_data[pitching_data['Test Type'] == test_type]

        if len(type_data) >= 2:
            most_recent_velo = type_data.iloc[-1]['Max Velo']
            previous_velo = type_data.iloc[-2]['Max Velo']
            all_time_velo = type_data['Max Velo'].max()
            today_change = most_recent_velo - previous_velo
            today_pct_change = (today_change / previous_velo) * 100 if previous_velo != 0 else 0
            all_time_change = most_recent_velo - type_data.iloc[0]['Max Velo']
            all_time_pct_change = (all_time_change / type_data.iloc[0]['Max Velo']) * 100 if type_data.iloc[0]['Max Velo'] != 0 else 0
            is_pr = " PR" if most_recent_velo == all_time_velo else ""

            today_color = 'black' if today_change == "N/A" else ('green' if today_change > 0 else 'red')
            all_time_color = 'black' if all_time_change == "N/A" else ('green' if all_time_change > 0 else 'red')

            velocity_changes.append([test_type, f"{most_recent_velo:.2f} MPH   {is_pr}",
                                     f"{today_change:+.2f} ({today_pct_change:+.1f}%)", today_color,
                                     f"{all_time_change:+.2f} ({all_time_pct_change:+.1f}%)", all_time_color, colors[i]])
        elif len(type_data) == 1:
            most_recent_velo = type_data.iloc[-1]['Max Velo']
            is_pr = " PR" if most_recent_velo == type_data['Max Velo'].max() else ""
            velocity_changes.append([test_type, f"{most_recent_velo:.2f} MPH   {is_pr}", "N/A", "black", "N/A", "black", colors[i]])
        else:
            velocity_changes.append([test_type, "N/A", "N/A", "black", "N/A", "black", colors[i]])

    fig = plt.figure(figsize=(14, 12))
    gs = fig.add_gridspec(nrows=3, ncols=1, height_ratios=[0.3, 1, 4])

    fig.suptitle(f"BEA Throwing Drill Velocity and Arm Score Over Time", fontsize=16, fontweight='bold', y=0.97)
    ax_table = fig.add_subplot(gs[1])
    ax_table.axis('off')

    col_labels = ['BEA Drill', 'Velo', 'Today', 'All Time']
    table_data = []
    for row in velocity_changes:
        formatted_velo = row[1].replace("PR", r"$\bf{PR}$")  # Bold PR
        table_data.append([row[0], formatted_velo, row[2], row[4]])

    table = ax_table.table(cellText=table_data, colLabels=col_labels, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    for (i, j), cell in table.get_celld().items():
        if i == 0:
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#D3D3D3')
        elif j == 0:
            cell.set_text_props(color=velocity_changes[i - 1][6])  # Throw type color
        elif j == 2:
            cell.set_text_props(color=velocity_changes[i - 1][3])  # Today color
        elif j == 3:
            cell.set_text_props(color=velocity_changes[i - 1][5])  # All time color

    ax1 = fig.add_subplot(gs[2])
    handles = []
    all_velo_values = []

    for test_type, color, marker in zip(test_types, colors, ['o', 'x', 's', 'D', '^', 'P', '*']):
        if test_type == 'Trackman Bullpen':
            test_data = trackman_data
        else:
            test_data = pitching_data[pitching_data['Test Type'] == test_type]

        if not test_data.empty:
            line_max, = ax1.plot(test_data['Date'], test_data['Max Velo'], color=color, marker=marker, label=f'{test_type} - Max Velo')
            handles.append((line_max, f'{test_type} - Max Velo'))
            all_velo_values.extend(test_data['Max Velo'])

             # **Annotate the last velocity score**
            last_entry = test_data.iloc[-1]
            ax1.text(last_entry['Date'], last_entry['Max Velo'] + 1, f"{last_entry['Max Velo']:.1f} MPH",
                    color=color, ha='center', fontsize=9, fontweight='bold')

    # Adjust Y-axis limits for better visibility
    if all_velo_values:
        min_velo = max(50, min(all_velo_values) * 0.9)  # Ensures the lowest plotted value is at least 50
        max_velo = max(all_velo_values) * 1.1  # Gives spacing at the top
        ax1.set_ylim(min_velo, max_velo)

    if not armcare_data.empty:
        bars = ax1.bar(armcare_data['Date'], armcare_data['Arm Score'], width=3, color='skyblue', alpha=0.7)
        handles.append((bars[0], 'Arm Score'))

        max_arm_score = armcare_data['Arm Score'].max()
        min_arm_score = armcare_data['Arm Score'].min() if len(armcare_data) > 1 else 50  # Prevents bars from plotting too low
        arm_score_limit = max(100, max_arm_score * 1.1)

        ax1.set_ylim(min(min_velo, min_arm_score * 0.9), max(max_velo, arm_score_limit))  # Unifies Y-axis scaling

            # **Highlight max arm score in gold**
        for bar, score in zip(bars, armcare_data['Arm Score']):
            if score == max_arm_score:
                bar.set_color('gold')  # Change to gold
                ax1.text(bar.get_x() + bar.get_width() / 2, score + 0.5, f"{score:.1f}", 
                        color='black', ha='center', fontsize=10, fontweight='bold')

    ax1.set_ylabel("Velocity (MPH) and Arm Score")
    ax1.yaxis.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

    if not armcare_data.empty:
        ax1.set_xticks(armcare_data['Date'])
        ax1.set_xticklabels(armcare_data['Date'].dt.strftime('%m/%d'), rotation=45, ha='right', fontsize=8)

    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)